# 统计标签信息

In [56]:
# !pip install openpyxl

In [61]:
import os
import pandas as pd
from tqdm import tqdm

root_dir = '/data/hjl/data'
label_file = 'wsi_label_info-20240905.xlsx'
label_path = os.path.join(root_dir, label_file)

# 读取xlsx文件
df = pd.read_excel(label_path)

# 去重 去错误图片 去没有检测结果

In [62]:
# 去重

print(len(df['wsi_id']))
# print(df['wsi_label'].value_counts())
print('########################################')

df = df.drop_duplicates(subset='wsi_id')
print(len(df['wsi_id']))
# print(df['wsi_label'].value_counts())




54367
########################################
49935


In [63]:
# 去掉错误图片
err_table = pd.read_excel('/data/hjl/data/tct_img_error.xlsx')
err_list = list(err_table['wsi_name'].unique())
err_list = [item + '.pt' for item in err_list]
print(len(df['wsi_id']))
df = df[~df['wsi_id'].isin(err_list)]
print(len(df['wsi_id']))


49935
49777


In [64]:
# 去掉没有检测特征
feat_dir = '/data/hjl/data/ori_img_feats/embed6/'
count = 0
print(len(df['wsi_id']))
for index, wsi in tqdm(df.iterrows()): 
    if not os.path.exists(os.path.join(feat_dir, wsi['wsi_id'].replace('.pt', ''))):
        count += 1
        # print(wsi['wsi_id'])
        df = df.drop(index)
print(len(df['wsi_id']))
print(count)
# print(df['wsi_label'].value_counts())
# print('########################################')

49777


49777it [33:30, 24.76it/s] 

49032
745


In [65]:
# 去掉没有检索到WSI文件夹
pd_path = pd.read_csv('/data/hjl/data/wsi_label_path.csv')
exist_list = list(pd_path['wsi_id'].unique())
print(len(df['wsi_id']))
df = df[df['wsi_id'].isin(exist_list)]
print(len(df['wsi_id']))


49032
49030


In [66]:
print(len(df['wsi_id']))
print(df['wsi_label'].value_counts())

49030
wsi_label
NILM            29351
ASC-US           5871
LSIL             4424
BV               3781
M                1802
ASC-H            1487
HSIL             1189
T                 587
AGC               497
E                  17
Actinomyces         8
NILM-atrophy        4
ASC-US+BV           2
炎症                  2
LSIL+BV             1
FXJ                 1
ASC-H+BV            1
ASC-h               1
？                   1
HSV                 1
ASCH                1
SCC                 1
Name: count, dtype: int64


In [67]:
df['wsi_label'] = df['wsi_label'].replace('ASCH', 'ASC-H')
df['wsi_label'] = df['wsi_label'].replace('ASC-h', 'ASC-H')
selected_values = ['NILM', 'ASC-US', 'LSIL', 'ASC-H', 'HSIL', 'AGC', 'T', 'M', 'BV']
df = df[df['wsi_label'].isin(selected_values)]
print(len(df['wsi_id']))
print(df['wsi_label'].value_counts())


48991
wsi_label
NILM      29351
ASC-US     5871
LSIL       4424
BV         3781
M          1802
ASC-H      1489
HSIL       1189
T           587
AGC         497
Name: count, dtype: int64


In [68]:
import random
volume1_path = '/data/hjl/data/TCTGC50k-labels/TCTGC50k-volume1-labels-path.csv'
volume1 = pd.read_csv(volume1_path)
volume1_id = list(volume1['wsi_id'])
all_id = list(df['wsi_id'])
print(len(all_id))
# 使用列表推导式从list1中减去list2的元素
all_id = [item for item in all_id if item not in volume1_id]
# 打印结果
print(len(all_id))
volume2_id = all_id[:10000]
volume3_id = all_id[10000:20000]
volume4_id = all_id[20000:30000]
volume5_id = all_id[30000:]
volumes_id = [[], volume1_id, volume2_id, volume3_id, volume4_id, volume5_id]
# print(len(volumes_id[5]))

48991
38991


In [70]:
output_dir = '/data/hjl/data/TCTGC50k-labels'
output_files = [f'TCTGC50k-volume{i}-labels-path.csv' for i in range(0, 6)]
path_csv = pd.read_csv(wsi_label_path)
for i in range(2,6):
    volume_id = volumes_id[i]
    output_df = []
    for id in volume_id:
        output_df.append(path_csv[path_csv['wsi_id'] == id])
    output_df = pd.concat(output_df)
    output_path = os.path.join(output_dir, output_files[i])
    output_df.to_csv(output_path, index=False)

# 创建软链接

In [7]:
import os
import pandas as pd
from tqdm import tqdm

label_dir = '/data/hjl/data/TCTGC50k-labels'
label_files = [f'TCTGC50k-volume{i}-labels-path.csv' for i in range(0, 6)]

output_dir = '/data/hjl/data/TCTGC50k'
output_sub_dirs = [f'TCTGC50k-volume{i}' for i in range(0, 6)]

for i in tqdm(range(2,6)):
    label_path = os.path.join(label_dir, label_files[i])
    df_volume = pd.read_csv(label_path)
    for index, row in df_volume.iterrows():
        wsi_name = row['wsi_id'].replace('.pt', '')
        wsi_img_path = row['wsi_img_path']
        os.symlink(wsi_img_path, os.path.join(output_dir, output_sub_dirs[i], wsi_name))



100%|██████████| 4/4 [00:43<00:00, 10.98s/it]


# 构造子样本标签

In [ ]:
# import numpy as np
# np.random.seed(42)

# root_dir = '/data/hjl/data'
# sample_num = 2000
# output_file = f'wsi_label_{sample_num}.csv'
# output_path = os.path.join(root_dir, output_file)
# # sample_df = df.sample(n=sample_num, replace=False)
# # sample_df.to_csv(output_path, index=False)
# discard_wsi_list = ['CX20193918-ASC-US.pt', 'T2303300641.pt']

# value_list = ['NILM', 'ASC-US', 'LSIL', 'ASC-H', 'HSIL', 'AGC', 'T', 'M', 'BV']
# specific_value = 'NILM-atrophy'  
# specific_samples = df[df['wsi_label'] == specific_value]

# # 计算特定值样本的数量
# specific_samples_count = specific_samples.shape[0]

# # 然后从剩余的数据中随机选择 2000 - specific_samples_count 个样本
# remaining_samples_count = 2000 - specific_samples_count

# df = df[~df['wsi_id'].isin(discard_wsi_list)]
# remaining_samples = df[df['wsi_label'].isin(value_list)].sample(n=remaining_samples_count, replace=False)

# # 将特定值样本和随机选择的样本合并
# combined_samples = pd.concat([specific_samples, remaining_samples])

# # 再次保存到CSV文件
# combined_samples.to_csv(output_path, index=False)



In [ ]:
# print(combined_samples['wsi_label'].value_counts())
# unique_samples = combined_samples.drop_duplicates()
# print('########')
# unique_samples['wsi_label'].value_counts()

wsi_label
NILM            1203
ASC-US           209
BV               177
LSIL             171
M                 86
ASC-H             59
HSIL              38
T                 32
AGC               21
NILM-atrophy       4
Name: count, dtype: int64
########


wsi_label
NILM            1203
ASC-US           209
BV               177
LSIL             171
M                 86
ASC-H             59
HSIL              38
T                 32
AGC               21
NILM-atrophy       4
Name: count, dtype: int64

In [ ]:
# import os
# import pandas as pd

# root_dir = '/data/hjl/data'
# label_file = 'wsi_label_2000.csv'
# label_path = os.path.join(root_dir, label_file)

# # 读取xlsx文件
# df = pd.read_csv(label_path)

# print(df['wsi_label'].describe())
# print(df['wsi_label'].value_counts())


count     2000
unique      10
top       NILM
freq      1203
Name: wsi_label, dtype: object
wsi_label
NILM            1203
ASC-US           209
BV               177
LSIL             171
M                 86
ASC-H             59
HSIL              38
T                 32
AGC               21
NILM-atrophy       4
Name: count, dtype: int64


## 匹配WSI位置

In [ ]:
# import os, glob
# import pandas as pd
# from tqdm import tqdm

# root_dir = '/data/hjl/data'
# label_file = 'wsi_label_2000.csv'
# label_path = os.path.join(root_dir, label_file)
# sample_num = 2000
# output_file = f'wsi_label_{sample_num}_path.csv'
# output_path = os.path.join(root_dir, output_file)
# wsi_root_dir = '/data2/px_data_lake/TCT_POS_NEG_DATA3'

# # 读取xlsx文件
# df = pd.read_csv(label_path)
# wsi_paths = []
# for index, wsi in tqdm(df.iterrows()): 
#     wsi_name = wsi['wsi_id'].replace('.pt', '')
#     wsi_path = glob.glob(os.path.join(wsi_root_dir, '*', '*', wsi_name), recursive=True)
#     if len(wsi_path) == 0:
#         print(f'No matching WSI found for {wsi_name}')
#         wsi_paths.append('flag')
#     else:
#         wsi_paths.append(wsi_path[0]) 
# df['wsi_path'] = wsi_paths
# df.to_csv(output_path, index=False)

2000it [00:08, 239.15it/s]


In [ ]:
# import os, glob
# import pandas as pd
# from tqdm import tqdm

# feat_dir = '/data/hjl/data/ori_img_feats/embed6/'
# root_dir = '/data/hjl/data'
# label_file = 'wsi_label_2000.csv'
# label_path = os.path.join(root_dir, label_file)
# df = pd.read_csv(label_path)
# count = 0
# for index, wsi in tqdm(df.iterrows()): 
#     wsi_id = wsi['wsi_id'].replace('.pt', '')
#     feat_path = glob.glob(os.path.join(feat_dir, wsi_id))
#     if len(feat_path) == 0:
#         print(f'No matching WSI found for {wsi_id}')
#         count += 1
# print(count)


2000it [00:00, 20395.10it/s]

0


# 统计图片信息

In [ ]:
# import os, glob
# import pandas as pd
# from tqdm import tqdm

# root_dir = '/data/hjl/data'
# label_file = 'wsi_label_2000_path.csv'
# label_path = os.path.join(root_dir, label_file)
# # sample_num = 2000
# output_file = f'wsi_label_2000_imgpath.csv'
# output_path = os.path.join(root_dir, output_file)
# # wsi_root_dir = '/data2/px_data_lake/TCT_POS_NEG_DATA3'

# # 读取xlsx文件
# df = pd.read_csv(label_path)
# wsi_paths = df['wsi_path']
# wsi_img_paths = []
# count = 0
# print(len(wsi_paths))
# for wsi_path in tqdm(wsi_paths):
#     tmp_path1 = os.path.join(wsi_path, 'TCT', 'OriginalImage')
#     tmp_path2 = os.path.join(wsi_path, 'DigitalSlice', 'OriginalImage')
#     tmp_path3 = os.path.join(wsi_path, 'OriginalImage')
#     tmp_path4 = 'flag'
#     flag = False
#     if os.path.exists(tmp_path1):
#         wsi_img_paths.append(tmp_path1)
#     elif os.path.exists(tmp_path2):
#         wsi_img_paths.append(tmp_path2)
#     elif os.path.exists(tmp_path3):
#         wsi_img_paths.append(tmp_path3)
#     else:
#         for filename in os.listdir(wsi_path):
#             if filename.endswith('.jpg'):
#                 flag = True
#         if flag:
#             wsi_img_paths.append(wsi_path)
#         else:
#             wsi_img_paths.append(tmp_path4)
#             print(wsi_path)
#             count += 1
# print(count)
# df['wsi_img_path'] = wsi_img_paths
# df.to_csv(output_path, index=False)

# # df['wsi_path'] = wsi_paths
# # df.to_csv(output_path, index=False)

2000


100%|██████████| 2000/2000 [00:05<00:00, 358.20it/s]

0


# 创建数据集软连接

In [ ]:
import os
import pandas as pd

output_dir = '/data/hjl/data/TCTGC-2000'
csv_file = '/data/hjl/data/wsi_label_2000_imgpath.csv'
df = pd.read_csv(csv_file)

for index, row in df.iterrows():
    wsi_name = row['wsi_id'].replace('.pt', '')
    wsi_img_path = row['wsi_img_path']
    os.symlink(wsi_img_path, os.path.join(output_dir, wsi_name))

